In [1]:
#Second test for git aliases (e.g., git-update-plus, git-next-checkin, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [2]:
# Global Setup
# Creating temp local repos to test on

TMP=/tmp/temp-git-test


git clone https://github.com/LorenzoFerraces/bash-test.git "$TMP"

# Move to second local to push some changes
command cd "$TMP" 
git pull origin main 
echo "hello second local" > hello.txt
git add hello.txt
git commit -m "first commit on second local"
git push


fatal: destination path '/tmp/temp-git-test' already exists and is not an empty directory.
From https://github.com/LorenzoFerraces/bash-test
 * branch            main       -> FETCH_HEAD
Already up to date.
bash: hello.txt: cannot overwrite existing file
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


In [3]:
# git-rename-file tests
# Assert file doesn't exist
[ ! -f "newHello.txt" ]; echo $?;

0


In [4]:
export GIT_MESSAGE="renamed hello.txt"
previous=$GIT_NO_CONFIRM
export GIT_NO_CONFIRM="1"
contents="$(cat hello.txt)"
git-rename-file hello.txt newHello.txt

issuing: git mv hello.txt newHello.txt
issuing: git add newHello.txt

GIT_NO_CONFIRM set to 1, so skipping confirmation
issuing: git commit -m 'renamed hello.txt'
    [main 45ee27f] renamed hello.txt
     1 file changed, 0 insertions(+), 0 deletions(-)
     rename hello.txt => newHello.txt (100%)
GIT_NO_CONFIRM set to 1, so skipping confirmation
issuing: git push --verbose
Pushing to https://github.com/LorenzoFerraces/bash-test.git
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 6 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (2/2), 279 bytes | 279.00 KiB/s, done.
Total 2 (delta 0), reused 1 (delta 0), pack-reused 0
POST git-receive-pack (452 bytes)
To https://github.com/LorenzoFerraces/bash-test.git
   44594a9..45ee27f  main -> main
updating local tracking ref 'refs/remotes/origin/main'
[main 45ee27f] renamed hello.txt
 1 file changed, 0 insertions(+), 0 deletions(-)
 rename hello.txt => newHello.txt (100%)

issuin

In [5]:
# Asserts
[ -f "newHello.txt" ] && [ "$(cat newHello.txt)" == "$(echo $contents)" ]; 
echo $?;

0


In [6]:
# git-restore-staged-alias test
echo "second change on local" >> newHello.txt
git add newHello.txt

In [7]:
# Assert theres is one staged file 
[ $(git status -s | grep -c "^[MTADRCU]") -eq 1 ]; echo $?; 

0


In [8]:
export GIT_MESSAGE="restored newHello.txt"
export GIT_NO_CONFIRM="1"
git-restore-staged-alias newHello.txt
rm -rf _git-trash

issuing: cp -vpf newHello.txt _git-trash
issuing: git restore --staged newHello.txt
'newHello.txt' -> '_git-trash/newHello.txt'
removed '_git-trash/hello.txt'
removed '_git-trash/newHello.txt'
removed directory '_git-trash'


In [9]:
# Assert theres are no staged files
[ $(git status -s | grep -c "^[MTADRCU]") -eq 0 ]; echo $?; 

0


In [10]:
# git-revert-file-alias tests
echo "second change on local" >> newHello.txt


In [11]:
# Assert theres is 1 unstaged file
[ $(git status -s | grep -c "^\s[MTADRCU]") -eq 1 ]; echo $?; 

0


In [12]:
#revert unstaged file
export GIT_MESSAGE="reverted newHello.txt"
export GIT_NO_CONFIRM="1"
git-revert-file-alias newHello.txt

issuing: cp -vpf newHello.txt _git-trash
issuing: git reset HEAD newHello.txt
issuing: git checkout -- newHello.txt
issuing: git stash list
'newHello.txt' -> '_git-trash/newHello.txt'
Unstaged changes after reset:
M	newHello.txt


In [13]:
# Assert file was reverted
# this matches any files except untracked
# read https://git-scm.com/docs/git-status#_short_formatz
[ $(git status -s | grep -c "^\s*[MTADRCU]") -eq 0 ]; echo $?; 


0


In [14]:
# undo changes and remove newly created files

# TODO2:
## also, converting unique ID's to codes like temp-git-test/.git/objects/pack/pack-HEX.idx
## EX: cat $log | perl -pe 's/\b[0-9]+\b/NUM/g; s/\b[a-f0-9]+\b/HEX/g;'
rm -f *.txt > /dev/null
git rm -f *.txt > /dev/null
git add . > /dev/null
git commit -m "cleanup test files"
git push

fatal: pathspec '_git-trash' did not match any files
[main f026d30] cleanup test files
 1 file changed, 1 deletion(-)
 delete mode 100644 newHello.txt
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 6 threads
Compressing objects: 100% (1/1), done.
Writing objects: 100% (2/2), 242 bytes | 242.00 KiB/s, done.
Total 2 (delta 0), reused 1 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   45ee27f..f026d30  main -> main


In [15]:
# delete test local repos

command cd "/tmp"
rm -rf temp-git-test > /dev/null
export GIT_NO_CONFIRM=$previous
